In [1]:
import os
import re
import csv
import nltk
import string
import PyPDF2
import requests
import builtins
import warnings
import wikipedia
import numpy as np
import unicodedata
import pandas as pd
from tqdm import tqdm
import language_tool_python
from bs4 import BeautifulSoup
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
warnings.filterwarnings("ignore")

### The following are helper methods used for Data Preprocessing

In [2]:
lang_tool = language_tool_python.LanguageTool('en-US')

In [3]:
def remove_non_utf8_chars(text):
    pattern = r'[^\x00-\x7F]+'  # Pattern to match non-UTF-8 characters
    return re.sub(pattern, ' ', text)

def grammar_check(text):
    matches = lang_tool.check(text)
    return len(matches)

def remove_encoded_chars(text):
    clean_text = ""
    for c in text:
        if not unicodedata.category(c) == 'Co':
            clean_text+=c
    return clean_text

def check_and_remove(text, tags):
    for tag in tags:
        if tag in text:
            text = text.replace(tag, " ")
    return text

In [4]:
def clean_text(text):

    tags_removed = ['caroline barrière', 'csi5180', 'winter 2024', 'csi4106' , 'fall 2023']
    
    translator = str.maketrans(" ", " ", string.punctuation)
    text = text.translate(translator)
    text = re.sub(r"\s+", " ", text).strip() # Remove extra spaces

    text = text.lower().strip()
    words = word_tokenize(text) 
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word) for word in words]
    text = " ".join(word for word in words)
    
    text = check_and_remove(text, tags_removed) # REMOVES COMMON OCCURRING TAGS
    text = remove_non_utf8_chars(text) # REMOVES NON UTF-8 CHRACTERS
    text = remove_encoded_chars(text)

    return text

In [5]:
def chunk_text(text, chunk_size=300):

    words = text.split()  # Split the text into a list of words
    chunk = []

    for word in words:
        chunk.append(word)
        if len(chunk) >= chunk_size:
            yield ' '.join(chunk)  # Yield a chunk as a joined string
            chunk = []  # Reset the chunk

    # Yield the final chunk (if any words are left)
    if chunk:
        yield ' '.join(chunk)

In [6]:
folder_path = './'
pdf_files = [f for f in os.listdir(folder_path) if f.endswith('.pdf')]
text_results = {}

### Extract textual data from the course PDFs.

In [7]:
def extract_pdf_text(pdf_file):
    
    with open(pdf_file, 'rb') as file:
        reader = PyPDF2.PdfReader(file)

        # Extract Text (page-by-page)
        text_data = []
        num_pages = len(reader.pages)
        for page_num in range(num_pages):
            page = reader.pages[page_num]
            page_text = page.extract_text()
            page_text = clean_text(page_text)
            
            if len(page_text.split()) > 20:
                text_data.append(page_text)
            else:
                text_data.append("")

        return text_data

In [8]:
for file_name in pdf_files:
    
    file_path = os.path.join(folder_path, file_name)
    try:
        text_data = extract_pdf_text(file_path)
        combined_text = " ".join(text_data)
        text_results[file_name] = combined_text
        
    except PyPDF2.errors.PdfReadError:
        print(f"Error processing file (pdferror): {file_name}")

In [9]:
pdf_text_dataframe = pd.DataFrame(columns=['text', 'text_length','gram_score'])


for texts in text_results.values():
    chunks = chunk_text(texts)
    
    for t in chunks:
        row = [t, len(t.split()) ,grammar_check(t)]
        pdf_text_dataframe.loc[len(pdf_text_dataframe)] = row
        

pdf_text_dataframe.to_csv("PDF_file_text_300.csv", index= False)

### Extract textual data from URLs referenced in the course PDFs.

In [10]:
pdf_urls = [] # Stores the URLs that point to a PDF file
forbidden_urls = [] # Keeps track of the URLs that could not be accessed

In [11]:
# Returns a list of all the unique URLs
def extract_urls(pdf_file):
    
    urls = []
    with open(pdf_file, 'rb') as file:
        
        reader = PyPDF2.PdfReader(file)
        num_pages = len(reader.pages)

        for page_num in range(num_pages):
            page = reader.pages[page_num]
            objects = page.get('/Annots', {})
            if not objects:
                continue
                
            for obj in objects:
                annot = obj.get_object()
                
                if annot['/Subtype'] == '/Link' and '/A' in annot:
                    if '/URI' in annot['/A']:
                        url = annot['/A'].get_object()['/URI']
                        urls.append(url)
                    elif '/S' in annot['/A'] and annot['/A']['/S'] == '/URI':    
                        url = annot['/A']['/URI']
                        urls.append(url)
                
    urls = list(set(urls)) # Avoids repetation of URLs
    return urls

In [12]:
# Gets texts from webpages and cleans it for processing and updates the the PDF URLs.
def extract_text_from_url(url):
    
    if is_pdf_url_headers(url):
        return ""
    else:
        text = extract_text_from_webpage(url)

    lines = (line.strip() for line in text.splitlines())
    chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
    text = " ".join(chunk for chunk in chunks if chunk)
    
    text = clean_text(text)
    return text

In [13]:
# Extracts and returns text from a webpage
def extract_text_from_webpage(url):
    
    extracted_text = " "
    try:
        response = requests.get(url, timeout=5)
        response.raise_for_status()  # Raise an exception for error status codes

        soup = BeautifulSoup(response.content, 'html.parser', from_encoding="iso-8859-1")
        for element in soup(["script", "style"]):
            element.extract() # Remove script and style elements as they often contain non-content text

        content_elements = soup.find_all(['p']) 
        for element in content_elements:
            extracted_text += element.get_text()
        
    except requests.exceptions.RequestException as e:
        print(f"Error fetching URL: {e}")
        forbidden_urls.append(url)
        extracted_text+= " "

    except requests.exceptions.Timeout as e:
        print(f"Error fetching URL: {e}")
        forbidden_urls.append(url)
        extracted_text+= " "
        
    return extracted_text

In [14]:
# Checks if the given URL points to a PDF or not
def is_pdf_url_headers(pdf_url):
    try:
        response = requests.head(pdf_url, timeout=5)
        content_type = response.headers.get('content-type')
        return content_type == 'application/pdf'
    
    except requests.exceptions.RequestException:
        return False 

In [15]:
# GET LIST OF ALL URLs
url_list = []

for file_name in tqdm(pdf_files):

    file_path = os.path.join(folder_path, file_name)
    try:
        urls = extract_urls(file_path)
        for i in urls:
            url_list.append(i)
    except PyPDF2.errors.PdfReadError:
        print(f"Error processing file (pdferror): {file_name}")

100%|███████████████████████████████████████████| 17/17 [00:00<00:00, 72.99it/s]


In [16]:
# GET TEXT FROM URLS
url_results = {}
url_dataframe = pd.DataFrame(columns=['text', 'text_length','gram_score'])


for url in tqdm(url_list):
    
    if is_pdf_url_headers(url):
        pdf_urls.append(url)
        continue
    
    article_texts = extract_text_from_url(url) # Input: URL; Output: Text retrieved from the link 
    chunks = chunk_text(article_texts)
    
    for t in chunks:
        row = [t, len(t.split()) ,grammar_check(t)]
        url_dataframe.loc[len(url_dataframe)] = row
            
        
url_dataframe.to_csv("URL_text_300.csv", index= False)

  1%|▎                                          | 2/249 [00:04<07:55,  1.92s/it]

Error fetching URL: 403 Client Error: Forbidden for url: https://www.datacamp.com/tutorial


  4%|█▌                                         | 9/249 [00:23<07:05,  1.77s/it]

Error fetching URL: 403 Client Error: Forbidden for url: https://www.researchgate.net/figure/Illustration-of-the-BERT-model-for-joint-intent-detection-and-slot-filling_fig3_359661835


  4%|█▋                                        | 10/249 [00:24<05:28,  1.38s/it]

...

 96%|███████████████████████████████████████▎ | 239/249 [13:15<00:14,  1.49s/it]

Error fetching URL: 403 Client Error: Forbidden for url: https://machinelearningmastery.com/a-gentle-introduction-to-positional-encoding-in-transformer-models-part-1/


 99%|████████████████████████████████████████▋| 247/249 [13:36<00:03,  1.99s/it]

Error fetching URL: 403 Client Error: Forbidden for url: https://deepchecks.com/5-approaches-to-solve-llm-token-limits/


100%|█████████████████████████████████████████| 249/249 [13:40<00:00,  3.30s/it]


### Extract textual data from URLs pointing to PDF files.

In [17]:
pdf_urls

['https://arxiv.org/pdf/2205.06573.pdf',
 'https://arxiv.org/pdf/2106.07447.pdf',
 'https://www.speech.kth.se/~rolf/NGSLT/presentations/ASV.pdf',
 'https://s3.amazonaws.com/assets.datacamp.com/production/course_3631/slides/chapter2.pdf',
 'https://arxiv.org/pdf/2106.07447.pdf',
 'https://cs.stanford.edu/~acoates/ba_dls_speech2016.pdf',
 'https://people.kth.se/~ghe/pubs/pdf/wagner2019speech.pdf',
 'https://nld.ict.usc.edu/cs644-spring2020/discussions/novikova-etal-emnlp2017.pdf']

In [18]:
# Extracts and returns text from URLs pointing towards PDF files. (Most likely research articles or reviews)
def extract_text_from_pdf_url(pdf_url):

    extracted_text = " "
    
    try:
        response = requests.get(pdf_url, timeout=5)
        pdf_file = open('temp.pdf', 'wb')  # Save PDF temporarily
        pdf_file.write(response.content)
        pdf_file.close()

        pdf_reader = PyPDF2.PdfReader('temp.pdf')
        num_pages = len(pdf_reader.pages)

        for page_num in range(num_pages):
            page = pdf_reader.pages[page_num]
            text = page.extract_text()
            extracted_text += text
            
    except requests.exceptions.RequestException as e:
        print(f"Error fetching URL: {e}")
        forbidden_urls.append(url)
        extracted_text+= " "

    except requests.exceptions.Timeout as e:
        print(f"Error fetching URL: {e}")
        forbidden_urls.append(url)
        extracted_text+= " "
    
    finally:
        if os.path.exists('temp.pdf'):  # Check if the file exists before deleting
            os.remove('temp.pdf')
    
    return extracted_text

In [19]:
PDF_url_dataframe = pd.DataFrame(columns=['text', 'text_length','gram_score'])

for url in tqdm(pdf_urls):
    
    text = extract_text_from_pdf_url(url)
    text = clean_text(text)
    
    chunks = chunk_text(text)
    
    for t in chunks:
        row = [t, len(t.split()) ,grammar_check(t)]
        PDF_url_dataframe.loc[len(PDF_url_dataframe)] = row
        
        
PDF_url_dataframe.to_csv("PDF_urls_text_300.csv", index= False, quoting=csv.QUOTE_NONE, escapechar=' ')        

100%|█████████████████████████████████████████████| 8/8 [01:59<00:00, 14.88s/it]


### Additional Data Extraction

#### 1. Extract textual data from Wikipedia articles corresponding to keywords of the course

In [20]:
# Manually gathered keywords that have an article related to it in correlation with Speech Processing
# This process can be further automated using keyword extraction.

queries = ['Virtual Assistant', 'Natural language generation' ,'Keyword spotting', 'Speech Recognition', 
           'Question answering', 'Speech Processing', 'Speech Synthesis', 'Voice activity detection',
           'utterance', 'paraphrase', 'Prosody (linguistics)' , 'Connectionist temporal classification', 
           'Language model', 'representation learning', 'word embedding', 'large language model',
          'Transformer (deep learning architecture)', 'Voice Analysis']

In [21]:
def get_wikipedia_content(search_query):

    try:
        result = wikipedia.search(search_query)[0]

        # Fetch the full article content
        page = wikipedia.page(result)
        return page.content

    except wikipedia.exceptions.DisambiguationError as e:
        print(f"Multiple results found: {e.options}")
        return None
    except wikipedia.exceptions.PageError:
        print(f"Page not found: {search_query}")
        return None

In [22]:
texts_keywords_wiki = {}

for keyword in queries:
    
    content = get_wikipedia_content(keyword)
    if content:
        texts_keywords_wiki[keyword] = clean_text(content)
    else:
        print("Wikipedia article not found or an error occurred.")

In [23]:

wiki_dataframe = pd.DataFrame(columns=['text', 'text_length','gram_score'])

for KW, content in tqdm(texts_keywords_wiki.items()):
    
    chunks = chunk_text(content)
    
    for t in chunks:
        row = [t, len(t.split()) ,grammar_check(t)]
        wiki_dataframe.loc[len(wiki_dataframe)] = row
        
        
wiki_dataframe.to_csv("Wiki_text_300.csv", index=False)

100%|███████████████████████████████████████████| 18/18 [01:01<00:00,  3.44s/it]


##### 2. Extract text from relevant chapters of the book, "Speech and Language Processing." by Daniel Jurafsky & James H. Martin.

In [24]:
book_folder = "./Book Chapters/"
chapters_pdf_files = [f for f in os.listdir(book_folder) if f.endswith('.pdf')]

In [25]:
results = {}

for file_name in chapters_pdf_files:
    
    file_path = os.path.join(book_folder, file_name)
    try:
        text_data = extract_pdf_text(file_path)
        combined_text = " ".join(text_data)
        results[file_name] = combined_text
    except PyPDF2.errors.PdfReadError:
        print(f"Error processing file (pdferror): {file_name}")

In [26]:
books_DF = pd.DataFrame(columns=['text', 'text_length','gram_score'])


for texts in results.values():
    chunks = chunk_text(texts)
    
    for t in chunks:
        t = clean_text(t)
        row = [t, len(t.split()) ,grammar_check(t)]
        books_DF.loc[len(books_DF)] = row
        

##### 3. Using the online book provided by Aalto Univeristy titled, "Introduction to Speech Processing"

In [27]:
extra_urls = ['https://speechprocessingbook.aalto.fi/Recognition_tasks_in_speech_processing.html',
              'https://speechprocessingbook.aalto.fi/Recognition/Voice_activity_detection.html',
             'https://speechprocessingbook.aalto.fi/Recognition/Speech_Recognition.html',
             'https://speechprocessingbook.aalto.fi/Recognition/Speaker_Recognition_and_Verification.html',
             'https://speechprocessingbook.aalto.fi/Recognition/Speaker_Diarization.html',
             'https://speechprocessingbook.aalto.fi/Recognition/Paralinguistic_speech_processing.html',
             'https://speechprocessingbook.aalto.fi/Speech_Synthesis.html',
             'https://speechprocessingbook.aalto.fi/Synthesis/Concatenative_speech_synthesis.html',
             'https://speechprocessingbook.aalto.fi/Synthesis/Statistical_parametric_speech_synthesis.html',
             'https://speechprocessingbook.aalto.fi/Analysis/Forensic_speaker_recognition.html']

In [28]:
# GET TEXT FROM URLS
extra_url_results = {}
extra_url_dataframe = pd.DataFrame(columns=['text', 'text_length','gram_score'])

for url in tqdm(extra_urls):
    
    article_texts = extract_text_from_url(url) # Input: URL; Output: Text retrieved from the link 
    chunks = chunk_text(article_texts)
    
    for t in chunks:
        row = [t, len(t.split()) ,grammar_check(t)]
        extra_url_dataframe.loc[len(extra_url_dataframe)] = row
        

100%|███████████████████████████████████████████| 10/10 [00:53<00:00,  5.37s/it]


In [29]:
new_DF = pd.concat([books_DF, extra_url_dataframe], ignore_index=True)
new_DF.to_csv("Book_text_300.csv", index=False, quoting=csv.QUOTE_NONE, escapechar=' ')

In [34]:
wiki_dataframe.iloc[0,:]

text           a virtual assistant va is a software agent tha...
text_length                                                  300
gram_score                                                    20
Name: 0, dtype: object